# Logistic Regression: Explanation of the model

Given a Observation $x = (1,~x_1, ~...~,~x_p)$ the model assings each $x_i$ the probability $P(y_i = 1 | x_i) = \pi(x_i)$ with

$$\begin{equation}
\pi(x) := \frac{e^{x'\beta}}{1 + e^{x'\beta}}
\end{equation}$$
where $\beta = (\beta_0,~\beta_1,~...~,\beta_p)$ are the parameters of the model.

The *logit* is given by

$$\begin{equation}
\operatorname{g}(x) := ln \frac{\pi(x)}{1 + \pi(x)} = x'\beta.
\end{equation}$$

The parameter $\beta$ is estimated with the likelihood function $l$:

$$
l(\beta) = \prod_{i = 1}^N \pi(x_i)^{y_i}[1-\pi(x_i)]^{1-y_i}
$$

with log likelihood
$$
L(\beta) = \ln(l(\beta)) = \sum_{i=1}^N \{y_i\ln(\pi(x_i) + (1-y_i)\ln(1- \pi(x_i)) \}
$$

Let's fit the first model. 

In [1]:
source("helpers.r")
df <- get_training_df()
fit  <- glm(target ~ ., data=df, family =binomial(link = "logit"))
summary(fit)

Warning message:
"package 'tidyverse' was built under R version 3.6.1"Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.1.1       v purrr   0.3.2  
v tibble  2.1.1       v dplyr   0.8.0.1
v tidyr   0.8.3       v stringr 1.4.0  
v readr   1.3.1       v forcats 0.4.0  
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
Warning message:
"package 'caret' was built under R version 3.6.1"Loading required package: lattice

Attaching package: 'caret'

The following object is masked from 'package:purrr':

    lift

Parsed with column specification:
cols(
  age = col_double(),
  sex = col_double(),
  cp = col_double(),
  trestbps = col_double(),
  chol = col_double(),
  fbs = col_double(),
  restecg = col_doubl


Call:
glm(formula = target ~ ., family = binomial(link = "logit"), 
    data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.9376  -0.3549   0.1385   0.4600   2.8127  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)            -0.3814076  3.8264252  -0.100 0.920601    
age                     0.0042520  0.0263861   0.161 0.871979    
thalach                 0.0096797  0.0118175   0.819 0.412727    
trestbps               -0.0044225  0.0132797  -0.333 0.739117    
oldpeak                -0.6844292  0.2758384  -2.481 0.013092 *  
ca                     -0.9410686  0.2547459  -3.694 0.000221 ***
chol                   -0.0008056  0.0045057  -0.179 0.858101    
restecgST-T_abnormalty  0.9117020  0.4398140   2.073 0.038179 *  
restecghypertrophy     -0.1768272  2.4561518  -0.072 0.942607    
fbsyes                  0.0585860  0.6347936   0.092 0.926467    
sexMale                -1.3420072  0.5909321  -2.271 0.023147

# Explanation of the R output
## Deviance Residuals


## Estimate
## Std. Error
## $z$ value and $P(>|z|)$
## Residual and null deviance

## Residual and null deviance

The deviance is defined as

$$
D = -2\ln(l(\hat{\beta}))
$$

Let's calculate the deviance for the whole model from scratch:

In [2]:
library(purrr)
pi <- predict(fit, df,  type="response")
y <- ifelse(df$target == "no_disease", 0, 1)
likelihood <- function(pi, y) pi ^ y * (1 - pi) ^ (1 - y)
(deviance <- -2 * log(prod(map2_dbl(pi, y, likelihood))))

[1] 153.6335

It is the same as the residual deviance of the summary of the fit. The null deviance is the deviance for the model containing only the intercept. This model estimates the probability for $y_i = 1$ with the average occurrence of $y = 1$:

In [3]:
N <- nrow(df)
df %>% count(target)
(pi_avg <- 132 / N)
pi <- rep(pi_avg, N)
(deviance <- -2 * log(prod(map2_dbl(pi, y, likelihood))))

target,n
no_disease,111
disease,132


[1] 0.5432099

[1] 335.0524

So $\pi(x_i) = 0.54$ for all $i = 1, ..., N$. You get the same result if you would fit a glm model, that contains only the intercept:

In [4]:
library(broom)
fit  <- glm(target ~ 1, data=df, family =binomial(link = "logit"))
beta_0 <- (tidy(fit))[2]
(pi_avg_fit <- exp(beta_0) / (1 + exp(beta_0)))

estimate
0.5432099


## AIC
## Number of Fisher Scoring iterations

## Fitting a model only using significant predictors

In [5]:
fit_sig  <- glm(target ~ oldpeak + ca + restecg + sex + cp, data=df, family =binomial(link = "logit"))
summary(fit_sig)


Call:
glm(formula = target ~ oldpeak + ca + restecg + sex + cp, family = binomial(link = "logit"), 
    data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.5178  -0.5177   0.2164   0.5578   2.2848  

Coefficients:
                       Estimate Std. Error z value Pr(>|z|)    
(Intercept)              1.1810     0.4950   2.386  0.01704 *  
oldpeak                 -1.0092     0.2171  -4.649 3.34e-06 ***
ca                      -0.8647     0.2055  -4.207 2.59e-05 ***
restecgST-T_abnormalty   0.7777     0.3790   2.052  0.04015 *  
restecghypertrophy      -0.7284     2.1290  -0.342  0.73224    
sexMale                 -1.3938     0.4486  -3.107  0.00189 ** 
cpatypical_angina        1.5919     0.5205   3.058  0.00223 ** 
cpnon-anginal_pain       2.5618     0.4811   5.325 1.01e-07 ***
cpasymptomatic           2.9029     0.7255   4.001 6.30e-05 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken

## Likelihood ratio test

In [6]:
library(lmtest)
lrtest(fit, fit_sig)

Warning message:
"package 'lmtest' was built under R version 3.6.1"Loading required package: zoo
Warning message:
"package 'zoo' was built under R version 3.6.1"
Attaching package: 'zoo'

The following objects are masked from 'package:base':

    as.Date, as.Date.numeric



#Df,LogLik,Df,Chisq,Pr(>Chisq)
1,-167.52622,NA,NA,NA
9,-93.42006,8,148.2123,4.626035e-28
